### This notebook demonstrates the use of scikit-learn metrics and logging them to Tensorboard using the _tf.summary_ API

In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from keras import layers

from sklearn.metrics import jaccard_score
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

2022-06-30 18:11:52.122250: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-30 18:11:52.122284: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class JaccardScoreCallback(keras.callbacks.Callback):
    def __init__(self, model, X_test, y_test, log_dir):
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
        self.keras_metric = tf.keras.metrics.Mean('jaccard_score')
        self.epoch = 0
        self.summary_writer = tf.summary.create_file_writer(
            os.path.join(log_dir, model.name))
            
    def on_epoch_end(self, batch, logs = None):
            self.epoch += 1
            self.keras_metric.reset_state()
            predictions = self.model.predict(self.X_test)
            jaccard_value = jaccard_score(np.argmax(predictions, axis = -1), 
                                          self.y_test, average = None)
            self.keras_metric.update_state(jaccard_value)
            self._write_metric(
                self.keras_metric.name, self.keras_metric.result().numpy().astype(float))
            
    def _write_metric(self, name, value):
            with self.summary_writer.as_default():
                tf.summary.scalar(name, value, step = self.epoch)
                self.summary_writer.flush()

In [3]:
NUM_CLASSES = 10
INPUT_SHAPE = (8, 8, 1)

digits = load_digits()
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42)

#Scale images.
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

#Reshape images.
X_train = np.reshape(X_train, (X_train.shape[0], 8, 8))
X_test = np.reshape(X_test, (X_test.shape[0], 8, 8))

X_train = np.expand_dims(X_train, axis = -1)
X_test = np.expand_dims(X_test, axis = -1)
X_train.shape, X_test.shape

((1437, 8, 8, 1), (360, 8, 8, 1))

In [4]:
#Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes = NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, num_classes = NUM_CLASSES)

In [5]:
model = keras.Sequential([
    keras.layers.Input(shape = INPUT_SHAPE),
    layers.Conv2D(filters = 32, kernel_size = (2, 2), activation = 'relu'),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

2022-06-30 18:12:00.437075: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-30 18:12:00.437128: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-30 18:12:00.437157: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debonair): /proc/driver/nvidia/version does not exist
2022-06-30 18:12:00.459350: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 7, 7, 32)          160       
                                                                 
 flatten (Flatten)           (None, 1568)              0         
                                                                 
 dropout (Dropout)           (None, 1568)              0         
                                                                 
 dense (Dense)               (None, 10)                15690     
                                                                 
Total params: 15,850
Trainable params: 15,850
Non-trainable params: 0
_________________________________________________________________


In [7]:
BATCH_SIZE = 4
EPOCHS = 15
CALLBACKS = [JaccardScoreCallback(model, X_test, np.argmax(y_test, axis = -1), 'logs')]

model.fit(X_train, y_train, batch_size = BATCH_SIZE,
          epochs = EPOCHS,
          callbacks = CALLBACKS, verbose = 1)

Epoch 1/15
360/360 [==============================] - 2s 2ms/step - loss: 2.0507 - accuracy: 0.4711
Epoch 2/15
360/360 [==============================] - 1s 2ms/step - loss: 1.0524 - accuracy: 0.8455
Epoch 3/15
360/360 [==============================] - 1s 2ms/step - loss: 0.5543 - accuracy: 0.8859
Epoch 4/15
360/360 [==============================] - 1s 2ms/step - loss: 0.4000 - accuracy: 0.9054
Epoch 5/15
360/360 [==============================] - 1s 2ms/step - loss: 0.3287 - accuracy: 0.9165
Epoch 6/15
360/360 [==============================] - 1s 2ms/step - loss: 0.2793 - accuracy: 0.9255
Epoch 7/15
360/360 [==============================] - 1s 2ms/step - loss: 0.2648 - accuracy: 0.9193
Epoch 8/15
360/360 [==============================] - 1s 2ms/step - loss: 0.2351 - accuracy: 0.9360
Epoch 9/15
360/360 [==============================] - 1s 2ms/step - loss: 0.2127 - accuracy: 0.9422
Epoch 10/15
360/360 [==============================] - 1s 2ms/step - loss: 0.2130 - accuracy: 0.9360